<a href="https://colab.research.google.com/github/ATIK2110018/Wind_netCDF/blob/main/netCDF_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install xarray matplotlib cartopy imageio -q

In [46]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import imageio
import os

ds = xr.open_dataset("/content/coast_wind_1.nc")
u = ds['u10']
v = ds['v10']
lat = ds['latitude']
lon = ds['longitude']
time = ds['valid_time']

output_folder = "wind_gif_frames"
os.makedirs(output_folder, exist_ok=True)

filenames = []
for i in range(len(time)):
    u_t = u[i, :, :]
    v_t = v[i, :, :]
    speed = np.sqrt(u_t**2 + v_t**2)

    fig = plt.figure(figsize=(16, 9), facecolor='black')
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([83.5, 95.5, 19.5, 27], crs=ccrs.PlateCarree())
    ax.set_facecolor("black")
    ax.coastlines(color='white', linewidth=0.25)
    ax.add_feature(cfeature.BORDERS, edgecolor='white', linewidth=0.25)
    ax.add_feature(cfeature.LAND, facecolor='black')
    ax.add_feature(cfeature.OCEAN, facecolor='black')

    gl = ax.gridlines(draw_labels=True, color='gray', alpha=0.5, linestyle='--', linewidth=0.5)
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'color': 'white'}
    gl.ylabel_style = {'color': 'white'}

    strm = ax.streamplot(
        lon, lat, u_t, v_t,
        color=speed.values,
        cmap='jet',
        linewidth=0.4,
        density=1.6,
        arrowsize=.8,
        transform=ccrs.PlateCarree()
    )

    timestamp = np.datetime_as_string(time[i].values, unit='h')
    ax.set_title(f"10m Wind Streamlines over Bangladesh \n{timestamp}",
                 fontsize=14, color='white', pad=20)

    cbar = fig.colorbar(strm.lines, ax=ax, orientation='vertical', shrink=0.7, pad=0.03)
    cbar.set_label("Wind Speed (m/s)", color='white')
    cbar.ax.tick_params(labelcolor='white')

    fname = os.path.join(output_folder, f"frame_{i:02d}.png")
    plt.savefig(fname, facecolor='black')
    filenames.append(fname)
    plt.close()

mp4_path = "coast_wind.mp4"
with imageio.get_writer(
    mp4_path,
    format="ffmpeg",
    mode="I",
    fps=1,
    codec="libx264",
    bitrate="8M"
) as writer:
    for filename in filenames:
        writer.append_data(imageio.imread(filename))

print(f"✅ MP4 saved to: {mp4_path}")


✅ MP4 saved to: coast_wind.mp4
